# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

In [2]:
data.info()

display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.1 Заполнение пропусков

In [3]:
display(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах присутствуют пропущенные значения в количестве 2174: days_employed и total_income.

Пропущены количественные значения в данных столбцах.

In [4]:
display(data.isna().sum()/data.shape[0])

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Согласно результату видим, что доля пропущенных значений в столбцах days_employed и total_income составляет 0.100999

In [5]:
display(data[data['total_income'].isna()].head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Согласно результату можно сделать вывод, что пропуски не зависят от других результатов по строкам, следовательно пропуски вероятно были случайные, для заполнения пропусков можно использовать медианные значения по группам income_type.

In [6]:
total_income_median = data['total_income'].median()

Выявляем медианное значение по столбцу 'total_income'.

In [7]:
data['total_income'] = data['total_income'].fillna(total_income_median)

Добавлены медианные значения вместо пропусков столбце 'income_type'.

In [8]:
display(data[data['total_income'].isna()].head()) #отсутствуют пропуски в столбце total_income

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


### Шаг 2.2 Проверка данных на аномалии и исправления.

Выявление аномалии в столбце 'children':

In [9]:
display(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Выявлены значения в данных: 20 и -1, судя по результатам это является отклонением и аномалием в таблице.
Заменяем данные значения:

In [10]:
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1

In [11]:
display(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Следующим шагом проверим столбец days_employed на аномалии (в данном столбце встречаются отрицательные значения - данные значения заменим на модульное значение, также стоит проверить насколько корректно введены дни в таблице) - и заменим пропуски Nan медианным значением после обработки на аномалии.

In [12]:
data.loc[data['days_employed'] < 0, 'days_employed'] = abs(data['days_employed']) #заменены отрицательные значения на положительные

In [13]:
data['days_employed'].sort_values(ascending=False).head(10) 

6954     401755.400475
10006    401715.811749
7664     401675.093434
2156     401674.466633
7794     401663.850046
4697     401635.032697
13420    401619.633298
17823    401614.475622
10991    401591.828457
8369     401590.452231
Name: days_employed, dtype: float64

В результате сортивровки, видим, что в столбце присутствуют значения с показателями более 1000 лет, 401755 дней / 365. Данные аномалии необходимо заменить на медианные значения.

In [14]:
days_employed_median = data['days_employed'].median()
data.loc[data['days_employed']/365 >= data['dob_years'], 'days_employed'] = days_employed_median
data['days_employed'].sort_values(ascending=False).head(10)

16335    18388.949901
4299     17615.563266
7329     16593.472817
17838    16264.699501
16825    16119.687737
3974     15835.725775
1539     15785.678893
4321     15773.061335
7731     15618.063786
15675    15410.040779
Name: days_employed, dtype: float64

In [15]:
display(data.loc[16335])

print('максимальный стаж:', data['days_employed'].max()/365, 'кол-во лет:', data.loc[16335, 'dob_years'])

children                                   1
days_employed                   18388.949901
dob_years                                 61
education                            среднее
education_id                               1
family_status                женат / замужем
family_status_id                           0
gender                                     F
income_type                        сотрудник
debt                                       0
total_income                   186178.934089
purpose             операции с недвижимостью
Name: 16335, dtype: object

максимальный стаж: 50.38068465909146 кол-во лет: 61


После изменения данных в столбце days_employed отсутствуют аномальные отклонения в стаже. максимальный стаж составляет в таблице 50 лет, при возрасте 61 год.

Далее произведем замену пропусков медианным значением.

In [16]:
data['days_employed'] = data['days_employed'].fillna(days_employed_median)

In [17]:
display(data[data['days_employed'].isna()].head()) #отсутствуют пропуски в столбце days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Финальная проперка на пропуски:

In [18]:
display(data.isna().sum()) #пропуски отсутствуют

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.3. Изменение типов данных.

In [19]:
data['total_income'] = data['total_income'].astype(int)

In [20]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


None

### Шаг 2.4. Удаление дубликатов.

In [21]:
print('Дубликатов в таблице:', data.duplicated().sum())

Дубликатов в таблице: 54


In [22]:
data = data.drop_duplicates()

print('Дубликатов в таблице:', data.duplicated().sum())

Дубликатов в таблице: 0


для поиска дубликатов использовал полностью таблицу, методом drop_duplicates() были удалены дубликаты из таблицы.

Возможные причины появления дубликатов:
1. подача заявки на кредит одним человеком несколько раз.
2. техническое дублирование заявок.
3. человеческий фактор при формировании списка.

Обработка неявных дубликатов в таблице.

Выявим какие уникальные значения в столбцах с категориальными значениями, где могут быть неявные дубликаты.

In [23]:
print(sorted(data['education'].unique()))

['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее', 'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']


In [24]:
print(sorted(data['family_status'].unique()))

['Не женат / не замужем', 'в разводе', 'вдовец / вдова', 'гражданский брак', 'женат / замужем']


In [25]:
print(sorted(data['income_type'].unique()))

['безработный', 'в декрете', 'госслужащий', 'компаньон', 'пенсионер', 'предприниматель', 'сотрудник', 'студент']


Таким образом, в столбцах education присутствуют неявные дубликаты.
Столбец education необходимо привести в один формат (строчные буквы).

In [26]:
data['education'] = data['education'].str.lower()
print(sorted(data['education'].unique()))

['высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создание двух новых датафреймов (словарей): 
1. каждому уникальному значению из education соответствует уникальное значение education_id.
2. каждому уникальному значению из family_status соответствует уникальное значение family_status_id.

In [27]:
education_data = data[['education', 'education_id']]
display(education_data.head())

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1


In [28]:
family_status_data = data[['family_status', 'family_status_id']]
display(family_status_data.head())

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1


Из исходной таблицы удаляем столбцы education и family_status:

In [29]:
data = data.drop(columns=['education', 'family_status'])
display(data.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,2194.220567,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создается столбец total_income_category с категориями:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [30]:
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    if total_income > 1000001:
        return 'A'

In [31]:
data['total_income_category'] = data['total_income'].apply(total_income_category)
display(data.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,2194.220567,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

Создаем новый столбец 'purpose_category' на основании столбца purpose, в который войдут следующие категории:
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

In [32]:
print(sorted(data['purpose'].unique()))

['автомобили', 'автомобиль', 'высшее образование', 'дополнительное образование', 'жилье', 'заняться высшим образованием', 'заняться образованием', 'на покупку автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'на проведение свадьбы', 'недвижимость', 'образование', 'операции с жильем', 'операции с коммерческой недвижимостью', 'операции с недвижимостью', 'операции со своей недвижимостью', 'покупка жилой недвижимости', 'покупка жилья', 'покупка жилья для сдачи', 'покупка жилья для семьи', 'покупка коммерческой недвижимости', 'покупка недвижимости', 'покупка своего жилья', 'получение высшего образования', 'получение дополнительного образования', 'получение образования', 'приобретение автомобиля', 'профильное образование', 'ремонт жилью', 'свадьба', 'свой автомобиль', 'сделка с автомобилем', 'сделка с подержанным автомобилем', 'строительство жилой недвижимости', 'строительство недвижимости', 'строительство собственной недвижимости', 'сыграть свадьбу']


In [33]:
def purpose_category_row(row):
    purpose = row['purpose']
    
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'недвижим' in purpose:
        return 'операции с недвижимостью'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'

In [34]:
data['purpose_category'] = data.apply(purpose_category_row, axis=1)
display(data.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,2194.220567,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [35]:
display(data['purpose_category'].value_counts())

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1:  Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [36]:
data_pivot_children = data.pivot_table(index=['children'], columns='debt', values='dob_years', aggfunc='count')
display(data_pivot_children)

debt,0,1
children,,
0,13044.0,1063.0
1,4411.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [37]:
data_pivot_children['ratio'] = data_pivot_children[1]/data_pivot_children[0]
display(data_pivot_children)

debt,0,1,ratio
children,,,
0,13044.0,1063.0,0.081493
1,4411.0,445.0,0.100884
2,1926.0,202.0,0.104881
3,303.0,27.0,0.089109
4,37.0,4.0,0.108108
5,9.0,NaN,NaN


In [38]:
#v1.расчет зависимости наличия задолженности у группы без детей: 
without_children_without_debt = data_pivot_children.loc[0, 0]
without_children_with_debt = data_pivot_children.loc[0, 1]
display(without_children_with_debt/(without_children_with_debt+without_children_without_debt))

0.07535266179910682

In [39]:
#v1.расчет зависимости наличия задолженности у группы с 1 ребенком:
one_children_without_debt = data_pivot_children.loc[1, 0]
one_children_with_debt = data_pivot_children.loc[1, 1]
display(one_children_with_debt/(one_children_with_debt+one_children_without_debt))

0.09163920922570017

In [40]:
#v1.расчет зависимости наличия задолженности у группы с 2 детьми:
two_children_without_debt = data_pivot_children.loc[2, 0]
two_children_with_debt = data_pivot_children.loc[2, 1]
display(two_children_with_debt/(two_children_with_debt+two_children_without_debt))

0.09492481203007519

In [41]:
#v1.расчет зависимости наличия задолженности у группы с 3 детьми:
three_children_without_debt = data_pivot_children.loc[3, 0]
three_children_with_debt = data_pivot_children.loc[3, 1]
display(three_children_with_debt/(three_children_with_debt+three_children_without_debt))

0.08181818181818182

In [42]:
#v1.расчет зависимости наличия задолженности у группы с 4 детьми:
four_children_without_debt = data_pivot_children.loc[4, 0]
four_children_with_debt = data_pivot_children.loc[4, 1]
display(four_children_with_debt/(four_children_with_debt+four_children_without_debt))

0.0975609756097561

##### Вывод 1: Согласно результату самая высокая доля по наличиям задолженности по возвратам кредита по группам количества детей у семей с 4,2,1 детьми - 9%. Но общий разброс не высокий, всего 2%, следовательно можно сделать вывод, что зависимость детей в семье не влияет на наличие задолженнсоти по кредитам. Семьи с 5 детьми не имели задолженности по возвратам кредита, но количество заявок с 5 детьми составляет менее 1% от общего количества заявок, не является репрезентативной группой.

##### Вопрос 2:  Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [43]:
data_pivot_family = data.pivot_table(index=['family_status_id'], columns='debt', values='dob_years', aggfunc='count')
display(data_pivot_family)

# женат / замужем 0
# гражданский брак 1
# вдовец / вдова 2
# в разводе 3
# Не женат / не замужем 4

debt,0,1
family_status_id,,
0,11413,931
1,3775,388
2,896,63
3,1110,85
4,2536,274


In [44]:
data_pivot_family['ratio'] = data_pivot_family[1]/data_pivot_family[0]
display(data_pivot_family)

debt,0,1,ratio
family_status_id,,,
0,11413,931,0.081574
1,3775,388,0.102781
2,896,63,0.070312
3,1110,85,0.076577
4,2536,274,0.108044


In [45]:
#v1.расчет зависимости наличия задолженности у группы женат/замужем (0):
zero_without_debt = data_pivot_family.loc[0, 0]
zero_with_debt = data_pivot_family.loc[0, 1]
display(zero_with_debt/(zero_with_debt+zero_without_debt))

0.07542125729099157

In [46]:
#v1.расчет зависимости наличия задолженности у группы гражданский брак (1):
first_without_debt = data_pivot_family.loc[1, 0]
first_with_debt = data_pivot_family.loc[1, 1]
display(first_with_debt/(first_with_debt+first_without_debt))

0.09320201777564256

In [47]:
#v1.расчет зависимости наличия задолженности у группы вдовец / вдова (2):
second_without_debt = data_pivot_family.loc[2, 0]
second_with_debt = data_pivot_family.loc[2, 1]
display(second_with_debt/(second_with_debt+second_without_debt))

0.06569343065693431

In [48]:
#v1.расчет зависимости наличия задолженности у группы в разводе (3):
third_without_debt = data_pivot_family.loc[3, 0]
third_with_debt = data_pivot_family.loc[3, 1]
display(third_with_debt/(third_with_debt+third_without_debt))

0.07112970711297072

In [49]:
#v1.расчет зависимости наличия задолженности у группы Не женат / не замужем (4):
fourth_without_debt = data_pivot_family.loc[4, 0]
fourth_with_debt = data_pivot_family.loc[4, 1]
display(fourth_with_debt/(fourth_with_debt+fourth_without_debt))

0.09750889679715302

##### Вывод 2: Согласно результату самая высокая доля наличия задолженности по возвратам кредита по группам семейного статуса у семей гражданского брака и не женатых/не замужних. Также доля в каждой группе не превышает 3%, что говорит о том, что влияние семейного статуса на наличие задолженности по кредитам отсутствует или минимальна.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [50]:
data_pivot_income = data.pivot_table(index=['total_income_category'], columns='debt', values='dob_years', aggfunc='count')
display(data_pivot_income)

#0–30.000 — 'E';
#30.001–50.000 — 'D';
#50.001–200.000 — 'C';
#200.001–1.000.000 — 'B';
#1.000.001 и выше — 'A'.

debt,0,1
total_income_category,,
A,23,2
B,4685,356
C,14673,1360
D,329,21
E,20,2


In [51]:
data_pivot_income['ratio'] = data_pivot_income[1]/data_pivot_income[0]
display(data_pivot_income)

debt,0,1,ratio
total_income_category,,,
A,23,2,0.086957
B,4685,356,0.075987
C,14673,1360,0.092687
D,329,21,0.063830
E,20,2,0.100000


In [52]:
#v1.расчет зависимости наличия задолженности у группы А:
А_without_debt = data_pivot_income.loc['A', 0]
А_with_debt = data_pivot_income.loc['A', 1]
display(А_with_debt/(А_with_debt+А_without_debt))

0.08

In [53]:
#v1.расчет зависимости наличия задолженности у группы B:
B_without_debt = data_pivot_income.loc['B', 0]
B_with_debt = data_pivot_income.loc['B', 1]
display(B_with_debt/(B_with_debt+B_without_debt))

0.07062090854989089

In [54]:
#v1.расчет зависимости наличия задолженности у группы C:
C_without_debt = data_pivot_income.loc['C', 0]
C_with_debt = data_pivot_income.loc['C', 1]
display(C_with_debt/(C_with_debt+C_without_debt))

0.08482504833780329

In [55]:
#v1.расчет зависимости наличия задолженности у группы D:
D_without_debt = data_pivot_income.loc['D', 0]
D_with_debt = data_pivot_income.loc['D', 1]
display(D_with_debt/(D_with_debt+D_without_debt))

0.06

In [56]:
#v1.расчет зависимости наличия задолженности у группы E:
E_without_debt = data_pivot_income.loc['E', 0]
E_with_debt = data_pivot_income.loc['E', 1]
display(E_with_debt/(E_with_debt+E_without_debt))

0.09090909090909091

##### Вывод 3: Согласно результату самая высокая доля наличия задолженности по возвратам кредита по группам дохода у групп Е и С. Самая многочисленная группа С, доля наличием задолженности по данной группе 8%, самая добросовестная группа с низкой долей наличия задолженности является группа D.  

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [57]:
data_pivot_category = data.pivot_table(index=['purpose_category'], columns='debt', values='dob_years', aggfunc='count')
display(data_pivot_category)

debt,0,1
purpose_category,,
операции с автомобилем,3905,403
операции с недвижимостью,10032,782
получение образования,3644,370
проведение свадьбы,2149,186


In [58]:
data_pivot_category['ratio'] = data_pivot_category[1]/data_pivot_category[0]
display(data_pivot_category)

debt,0,1,ratio
purpose_category,,,
операции с автомобилем,3905,403,0.103201
операции с недвижимостью,10032,782,0.077951
получение образования,3644,370,0.101537
проведение свадьбы,2149,186,0.086552


In [59]:
#v1.расчет зависимости наличия задолженности у группы операции с автомобилем:
auto_without_debt = data_pivot_category.loc['операции с автомобилем', 0]
auto_with_debt = data_pivot_category.loc['операции с автомобилем', 1]
display(auto_with_debt/(auto_with_debt+auto_without_debt))

0.09354688950789229

In [60]:
#v1.расчет зависимости наличия задолженности у группы операции с недвижимостью:
estate_without_debt = data_pivot_category.loc['операции с недвижимостью', 0]
estate_with_debt = data_pivot_category.loc['операции с недвижимостью', 1]
display(estate_with_debt/(estate_with_debt+estate_without_debt))

0.0723136674680969

In [61]:
#v1.расчет зависимости наличия задолженности у группы получение образования:
education_without_debt = data_pivot_category.loc['получение образования', 0]
education_with_debt = data_pivot_category.loc['получение образования', 1]
display(education_with_debt/(education_with_debt+education_without_debt))

0.09217737917289487

In [62]:
#v1.расчет зависимости наличия задолженности у группы проведение свадьбы:
wedding_without_debt = data_pivot_category.loc['проведение свадьбы', 0]
wedding_with_debt = data_pivot_category.loc['проведение свадьбы', 1]
display(wedding_with_debt/(wedding_with_debt+wedding_without_debt))

0.07965738758029979

##### Вывод 4: Самая добросовестная группа по целевым категориям кредита является группа по операциям с недвижимостью, доля в 9% по наличиям невозвратов кредитов группы по операциям с автомобилями и получением образования. Разброс по долям невысокий.

In [63]:
#код ревьюера
data.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
children,,,
0,14107,1063,0.075353
1,4856,445,0.091639
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


In [64]:
#код ревьюера
children_pivot = data.pivot_table(index=['children'], values='debt', aggfunc=['sum', 'count'])
children_pivot = children_pivot['sum'] / children_pivot['count']
children_pivot

,debt
children,
0,0.075353
1,0.091639
2,0.094925
3,0.081818
4,0.097561
5,0.000000


## Общий вывод:

### Общий вывод: согласно полученным результатам, влияние на наличие задолженности по кредитам, по выделенным группам в анализе, минимальное, в отношении доли наличия или отсутствия задолженности.
### Для более реалистичного анализа необходимо получить данные в средних суммах кредитов по группам, чтобы определить долю в суммарном отношении недобросовестных заемщиков, например, по операциям с недвижимостью суммарная доля невозмещаемых кредитов должна быть выше суммарной доли группы на проведение свадьбы, что может послужить более достоверным выводом о рисках для выдачи кредитов по определенным группам и целям кредита.    